## Импорты

In [28]:
from Cryptodome.Signature import PKCS1_v1_5
from Cryptodome.Hash import SHA256
from Cryptodome.PublicKey import RSA
from base64 import decodebytes, encodebytes

import json
import requests
import urllib.parse

import numpy as np
import pandas as pd

## Чтение из файла

In [29]:
filename = "./TSiS/blockchain3/test_data_100.csv"

df = pd.read_csv(filename, delimiter=';')
print(df.head())

       x1        x2         y
0 -0.2999 -0.200000  0.599161
1 -0.2996 -0.158579  0.600248
2 -0.2991 -0.126795  0.601089
3 -0.2984 -0.100000  0.601804
4 -0.2975 -0.076393  0.602441


In [30]:
df_x = df[["x1", "x2"]]
print(df_x.head())

       x1        x2
0 -0.2999 -0.200000
1 -0.2996 -0.158579
2 -0.2991 -0.126795
3 -0.2984 -0.100000
4 -0.2975 -0.076393


In [31]:
df_y = df["y"]
print(df_y.head())

0    0.599161
1    0.600248
2    0.601089
3    0.601804
4    0.602441
Name: y, dtype: float64


## Чтение ключей

In [32]:
privateKey = RSA.importKey(open("./TSiS/blockchain3/private.txt", "r").read())
publicKey = RSA.importKey(open("./TSiS/blockchain3/public.txt", "r").read())

publicKeyHex = publicKey.export_key('DER').hex()
privateKeyHex = privateKey.export_key('DER').hex()
print(publicKeyHex)
print(privateKeyHex)

30819f300d06092a864886f70d010101050003818d00308189028181009ce497586e7077737e8a8d0db9050b4ab59dfd0c9d19002881ee3ad97c9e2db61b7f8c36b7e3172f0cfb9f7c3227dee0b8260bdc8036c940a2cc38672d1a3b6d1a2352145e3bd1a34e9249aa921ea1657f2401558be630ec504527cafca0847cc265682cb813fb6c005c9d9a40b2accc53b66a12c66dcaabb747dd3c6335c73b0203010001
3082025b020100028181009ce497586e7077737e8a8d0db9050b4ab59dfd0c9d19002881ee3ad97c9e2db61b7f8c36b7e3172f0cfb9f7c3227dee0b8260bdc8036c940a2cc38672d1a3b6d1a2352145e3bd1a34e9249aa921ea1657f2401558be630ec504527cafca0847cc265682cb813fb6c005c9d9a40b2accc53b66a12c66dcaabb747dd3c6335c73b02030100010281803fd2adce877eb0d906cd0dfc5b1bb6303a197eeb36ad08e771939bd50560c36fa558098de8d834822b61e0cb010c3f1a634ef6dc483c013a8dff4da28b3274a1951fadbc71e4d9cfc17c956225d613a60fd11c7e93e2f7f36a8bb55a807ba3e596bae6fc84959a426344593142fec9b37d43cfe72f8c11d3bb62d899570de841024100e1b101eaada0581c72d6b295bf0bf202430b061c4b86e03d88c6b3fe9019f849b2ac8b59218254fcdde7e11d176d39a546a8ed6eeeda3c4f9a43219

## Подбор весов случайным подбором

In [33]:
data = {
    "w11": str(np.random.rand(1)[0]),
    "w12": str(np.random.rand(1)[0]),
    "w21": str(np.random.rand(1)[0]),
    "w22": str(np.random.rand(1)[0]),

    "v11": str(np.random.rand(1)[0]),
    "v12": str(np.random.rand(1)[0]),
    "v13": str(np.random.rand(1)[0]),
    "v21": str(np.random.rand(1)[0]),
    "v22": str(np.random.rand(1)[0]),
    "v23": str(np.random.rand(1)[0]),
    
    "w1": str(np.random.rand(1)[0]),
    "w2": str(np.random.rand(1)[0]),
    "w3": str(np.random.rand(1)[0]),
    "publickey": publicKeyHex
}

dataStr = json.dumps(data)
print(dataStr)

{"w11": "0.40711772012400627", "w12": "0.6628701855635437", "w21": "0.5301130836682106", "w22": "0.4287960791387946", "v11": "0.19605644690488266", "v12": "0.19652032330775804", "v13": "0.03220884373862021", "v21": "0.5140799676889524", "v22": "0.2953155111542447", "v23": "0.10407023733474596", "w1": "0.8202758384693493", "w2": "0.23438749826897243", "w3": "0.9815236538460907", "publickey": "30819f300d06092a864886f70d010101050003818d00308189028181009ce497586e7077737e8a8d0db9050b4ab59dfd0c9d19002881ee3ad97c9e2db61b7f8c36b7e3172f0cfb9f7c3227dee0b8260bdc8036c940a2cc38672d1a3b6d1a2352145e3bd1a34e9249aa921ea1657f2401558be630ec504527cafca0847cc265682cb813fb6c005c9d9a40b2accc53b66a12c66dcaabb747dd3c6335c73b0203010001"}


In [34]:
import math

def f(x):
    return 1 / (1 + math.exp(-x))

limit = 0.01

for k in range(0, 500000):
    data = {
        "w11": str(np.random.rand(1)[0]),
        "w12": str(np.random.rand(1)[0]),
        "w21": str(np.random.rand(1)[0]),
        "w22": str(np.random.rand(1)[0]),

        "v11": str(np.random.rand(1)[0]),
        "v12": str(np.random.rand(1)[0]),
        "v13": str(np.random.rand(1)[0]),
        "v21": str(np.random.rand(1)[0]),
        "v22": str(np.random.rand(1)[0]),
        "v23": str(np.random.rand(1)[0]),
        
        "w1": str(np.random.rand(1)[0]),
        "w2": str(np.random.rand(1)[0]),
        "w3": str(np.random.rand(1)[0]),
        "publickey": publicKeyHex
    }

    s = 0
    for i in range(0, len(df)):
        x1 = df_x['x1'][i]
        x2 = df_x['x2'][i]
        h11 = f(x1*float(data['w11']) + x2*float(data['w21']))
        h12 = f(x1*float(data['w12']) + x2*float(data['w22']))
        q1 = f(h11*float(data['v11']) + h12*float(data['v21']))*float(data['w1'])
        q2 = f(h11*float(data['v12']) + h12*float(data['v22']))*float(data['w2'])
        q3 = f(h11*float(data['v13']) + h12*float(data['v23']))*float(data['w3'])
        er = f(q1 + q2 + q3)
        
        s += (er - df_y[i]) * (er - df_y[i])

    if(s < limit):
        print(f"success: {s}")
        data["e"] = s
        break

dataStr = json.dumps(data)
print(dataStr)

success: 0.00855336939181472
{"w11": "0.8716569998131201", "w12": "0.6036557539698342", "w21": "0.6781071403196672", "w22": "0.22195912096588444", "v11": "0.7135882548573835", "v12": "0.9080348054517081", "v13": "0.9257711417285412", "v21": "0.38555194799069703", "v22": "0.49209711510293397", "v23": "0.659157483629762", "w1": "0.185458826826165", "w2": "0.3299425870525666", "w3": "0.17858636638818115", "publickey": "30819f300d06092a864886f70d010101050003818d00308189028181009ce497586e7077737e8a8d0db9050b4ab59dfd0c9d19002881ee3ad97c9e2db61b7f8c36b7e3172f0cfb9f7c3227dee0b8260bdc8036c940a2cc38672d1a3b6d1a2352145e3bd1a34e9249aa921ea1657f2401558be630ec504527cafca0847cc265682cb813fb6c005c9d9a40b2accc53b66a12c66dcaabb747dd3c6335c73b0203010001", "e": 0.00855336939181472}


## Публичный ключ

In [35]:
data["publickey"] = publicKeyHex

## Получение последнего блока и его хеша

In [36]:
resp = requests.get("http://89.108.115.118/nbc/chain")
lastBlock = resp.json()[-1]
print(lastBlock)

{'prevhash': '04c2b0c7f37e065beab29be91f7d571087743c404e26d3911fd4ec37d44f5080', 'data': {'w11': '0.613304195633', 'w12': '0.390071360605', 'w21': '0.703192660823', 'w22': '0.456601837993', 'v11': '0.141540698763', 'v12': '0.029720394515', 'v13': '0.855101767803', 'v21': '0.205988936748', 'v22': '0.177878092032', 'v23': '0.983949000097', 'w1': '0.451711939543', 'w2': '0.157189183348', 'w3': '0.234710304119', 'e': '0.009886374184', 'publickey': '30819f300d06092a864886f70d010101050003818d003081890281810082eca3805a75a0a718574dab93f92726a426c9452c5d6611cb9a5cd0e3fdbf0241a508853b7963c3d4d9f193dca6bcf75339423659a9f3122fa4e46d45e80a8459d8a20539418eaf68faf42bdbf8bcf88c20eda4cd1595ed82f696b90bcb4e120c96ec2f8e99aaec87399517d0ee1702545363fc11ea874cc1953daf6861dd490203010001'}, 'signature': '171c3ab3aa65102640917ae8298d28bc5c1d5546f9bfc3e35790d5726a4e0f1bf2898728eb1167c37758f9494bb3271aae1c06281ceff3479e2e04420da17753e487880deb9c173fee4998ece4622bf8c96b4bfbd703663439ffb83de02a287fc9621eaa9a4099633

In [37]:
def calculatePrevHash(data):
    return SHA256.new(
        bytearray.fromhex(data['prevhash']) + 
        str.encode(json.dumps(data['data']).replace(" ", "")) + 
        bytearray.fromhex(data['signature'])
    ).hexdigest()

In [38]:
calculatePrevHash(lastBlock)

'690725e5d170b55485fbcb713c792668fc31b2fbff22965f51a9f9363f5dd45f'

## Вычисления подписи строки с помощью алгоритма RSAwithSHA256

In [39]:
def calculateDataHash(data):
    return SHA256.new(str.encode(data))

def getSign(data, privateKey):
    signer = PKCS1_v1_5.new(privateKey)
    return signer.sign(calculateDataHash(data)).hex()

## Подготовка данных для отправки

In [40]:
def prettyFloat(x):
    return '%.12f' % float(x)

data2 = {
    "w11": prettyFloat(data["w11"]),
    "w12": prettyFloat(data["w12"]),
    "w21": prettyFloat(data["w21"]),
    "w22": prettyFloat(data["w22"]),
    "v11": prettyFloat(data["v11"]),
    "v12": prettyFloat(data["v12"]),
    "v13": prettyFloat(data["v13"]),
    "v21": prettyFloat(data["v21"]),
    "v22": prettyFloat(data["v22"]),
    "v23": prettyFloat(data["v23"]),
    "w1":  prettyFloat(data["w1"]),
    "w2":  prettyFloat(data["w2"]),
    "w3":  prettyFloat(data["w3"]),
    "e":  prettyFloat(data["e"]),
    "publickey": data["publickey"]
}
print(data2)

{'w11': '0.871656999813', 'w12': '0.603655753970', 'w21': '0.678107140320', 'w22': '0.221959120966', 'v11': '0.713588254857', 'v12': '0.908034805452', 'v13': '0.925771141729', 'v21': '0.385551947991', 'v22': '0.492097115103', 'v23': '0.659157483630', 'w1': '0.185458826826', 'w2': '0.329942587053', 'w3': '0.178586366388', 'e': '0.008553369392', 'publickey': '30819f300d06092a864886f70d010101050003818d00308189028181009ce497586e7077737e8a8d0db9050b4ab59dfd0c9d19002881ee3ad97c9e2db61b7f8c36b7e3172f0cfb9f7c3227dee0b8260bdc8036c940a2cc38672d1a3b6d1a2352145e3bd1a34e9249aa921ea1657f2401558be630ec504527cafca0847cc265682cb813fb6c005c9d9a40b2accc53b66a12c66dcaabb747dd3c6335c73b0203010001'}


In [41]:
dataStr = json.dumps(data2).replace(" ", "")
dataStr

'{"w11":"0.871656999813","w12":"0.603655753970","w21":"0.678107140320","w22":"0.221959120966","v11":"0.713588254857","v12":"0.908034805452","v13":"0.925771141729","v21":"0.385551947991","v22":"0.492097115103","v23":"0.659157483630","w1":"0.185458826826","w2":"0.329942587053","w3":"0.178586366388","e":"0.008553369392","publickey":"30819f300d06092a864886f70d010101050003818d00308189028181009ce497586e7077737e8a8d0db9050b4ab59dfd0c9d19002881ee3ad97c9e2db61b7f8c36b7e3172f0cfb9f7c3227dee0b8260bdc8036c940a2cc38672d1a3b6d1a2352145e3bd1a34e9249aa921ea1657f2401558be630ec504527cafca0847cc265682cb813fb6c005c9d9a40b2accc53b66a12c66dcaabb747dd3c6335c73b0203010001"}'

## Отправка блока

In [42]:
body = {
    "prevhash": calculatePrevHash(lastBlock),
    "data": data2,
    "signature": getSign(dataStr, privateKey)
}

bodyStr = json.dumps(body).replace(" ", "")
print(body)
print(bodyStr)

{'prevhash': '690725e5d170b55485fbcb713c792668fc31b2fbff22965f51a9f9363f5dd45f', 'data': {'w11': '0.871656999813', 'w12': '0.603655753970', 'w21': '0.678107140320', 'w22': '0.221959120966', 'v11': '0.713588254857', 'v12': '0.908034805452', 'v13': '0.925771141729', 'v21': '0.385551947991', 'v22': '0.492097115103', 'v23': '0.659157483630', 'w1': '0.185458826826', 'w2': '0.329942587053', 'w3': '0.178586366388', 'e': '0.008553369392', 'publickey': '30819f300d06092a864886f70d010101050003818d00308189028181009ce497586e7077737e8a8d0db9050b4ab59dfd0c9d19002881ee3ad97c9e2db61b7f8c36b7e3172f0cfb9f7c3227dee0b8260bdc8036c940a2cc38672d1a3b6d1a2352145e3bd1a34e9249aa921ea1657f2401558be630ec504527cafca0847cc265682cb813fb6c005c9d9a40b2accc53b66a12c66dcaabb747dd3c6335c73b0203010001'}, 'signature': '94fe18de1633c26920b8b8138ca16bf5d4f04babd565f7a8bd86fc1466f5303303fd10cc0dd883fcac17a0f3f76a21b207129d17ccdcdcb731f9aadc5748066d2ee1125d6f95ed898900c852ed1c46e2b75dfb5fd9ebd2873edbbaf3870290bf735bfe705bb4309a4

In [43]:
headers = {"Content-Type": "application/json;charset=UTF-8"}
# закомментировал, чтобы случайно не отправить
# resp = requests.post(f"http://89.108.115.118/nbc/newblock", headers=headers, json = body)
# print(resp.json())

{'status': 0, 'statusString': '', 'block': {'prevhash': '690725e5d170b55485fbcb713c792668fc31b2fbff22965f51a9f9363f5dd45f', 'data': {'w11': '0.871656999813', 'w12': '0.603655753970', 'w21': '0.678107140320', 'w22': '0.221959120966', 'v11': '0.713588254857', 'v12': '0.908034805452', 'v13': '0.925771141729', 'v21': '0.385551947991', 'v22': '0.492097115103', 'v23': '0.659157483630', 'w1': '0.185458826826', 'w2': '0.329942587053', 'w3': '0.178586366388', 'e': '0.008553369392', 'publickey': '30819f300d06092a864886f70d010101050003818d00308189028181009ce497586e7077737e8a8d0db9050b4ab59dfd0c9d19002881ee3ad97c9e2db61b7f8c36b7e3172f0cfb9f7c3227dee0b8260bdc8036c940a2cc38672d1a3b6d1a2352145e3bd1a34e9249aa921ea1657f2401558be630ec504527cafca0847cc265682cb813fb6c005c9d9a40b2accc53b66a12c66dcaabb747dd3c6335c73b0203010001'}, 'signature': '94fe18de1633c26920b8b8138ca16bf5d4f04babd565f7a8bd86fc1466f5303303fd10cc0dd883fcac17a0f3f76a21b207129d17ccdcdcb731f9aadc5748066d2ee1125d6f95ed898900c852ed1c46e2b75dfb

## Отправка автора

In [44]:
authorStr = "Сагадеев Артем, 11-909"
author = {
    "autor": authorStr,
    "sign": getSign(authorStr, privateKey),
    "publickey": publicKeyHex
}

headers = {"Content-Type": "application/json;charset=UTF-8"}
# закомментировал, чтобы случайно не отправить
# resp = requests.post(f"http://89.108.115.118/nbc/autor", headers=headers, json=author)
# print(resp.json())

{'status': 0, 'statusString': 'Автор успешно добавлен'}
